In [2]:
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
pandas2ri.activate()
base = importr('base')
stats = importr("stats")
utils = importr('utils')

import bbgclient
import dfutils
import datetime
import pandas as pd
import json
import numpy as np
import statsmodels.formula.api as sm

In [4]:
def multiple_underlying_df(ticker, start_date_yyyymmdd, end_date_yyyymmdd, api_host, fperiod):
    
    def last_elem_or_null(ts):
        if ts is None: return None
        if len(ts) == 0: return None
        return ts.iloc[-1]

    px = last_elem_or_null(bbgclient.bbgclient.get_timeseries(ticker,'PX_LAST',start_date_yyyymmdd,end_date_yyyymmdd,api_host=api_host))
    mkt_cap = last_elem_or_null(bbgclient.bbgclient.get_timeseries(ticker,'CUR_MKT_CAP',start_date_yyyymmdd,end_date_yyyymmdd,api_host=api_host))
    ev_component = last_elem_or_null(bbgclient.bbgclient.get_timeseries(ticker,'CUR_EV_COMPONENT',start_date_yyyymmdd,end_date_yyyymmdd,api_host=api_host))
    eqy_sh_out = last_elem_or_null(bbgclient.bbgclient.get_timeseries(ticker,'EQY_SH_OUT',start_date_yyyymmdd,end_date_yyyymmdd,api_host=api_host))
    best_ebitda = last_elem_or_null(bbgclient.bbgclient.get_timeseries(ticker,'BEST_EBITDA',start_date_yyyymmdd,end_date_yyyymmdd,{'BEST_FPERIOD_OVERRIDE':fperiod},api_host))
    best_sales = last_elem_or_null(bbgclient.bbgclient.get_timeseries(ticker,'BEST_SALES',start_date_yyyymmdd,end_date_yyyymmdd,{'BEST_FPERIOD_OVERRIDE':fperiod},api_host))
    best_eps = last_elem_or_null(bbgclient.bbgclient.get_timeseries(ticker,'BEST_EPS',start_date_yyyymmdd,end_date_yyyymmdd,{'BEST_FPERIOD_OVERRIDE':fperiod},api_host))
    div_ind_yield = last_elem_or_null(bbgclient.bbgclient.get_timeseries(ticker,'DIVIDEND_INDICATED_YIELD',start_date_yyyymmdd,end_date_yyyymmdd,api_host=api_host))
    best_opp = last_elem_or_null(bbgclient.bbgclient.get_timeseries(ticker,'BEST_OPP',start_date_yyyymmdd,end_date_yyyymmdd,api_host=api_host))
    best_ni = last_elem_or_null(bbgclient.bbgclient.get_timeseries(ticker,'BEST_NET_INCOME',start_date_yyyymmdd,end_date_yyyymmdd, {'BEST_FPERIOD_OVERRIDE':fperiod}, api_host=api_host))
    best_capex = last_elem_or_null(bbgclient.bbgclient.get_timeseries(ticker,'BEST_CAPEX',start_date_yyyymmdd,end_date_yyyymmdd,api_host=api_host))

    cols = ['Date','PX','CUR_MKT_CAP','EQY_SH_OUT','BEST_EBITDA','BEST_SALES',
            'BEST_EPS','DIVIDEND_INDICATED_YIELD','BEST_OPP','BEST_NET_INCOME','BEST_CAPEX','CUR_EV_COMPONENT']
    datum = [(pd.to_datetime(end_date_yyyymmdd),px,mkt_cap,eqy_sh_out,best_ebitda,best_sales,best_eps,
                                          div_ind_yield,best_opp,best_ni,best_capex,ev_component)]
    df = pd.DataFrame(columns=cols,data=datum)

    return df

In [5]:
def multiples_df(ticker, start_date_yyyymmdd, unaffected_date_yyyymmdd, api_host, fperiod, multiples_to_query='ALL'):
    if multiples_to_query == 'ALL':
        multiples_to_query = ['EV/EBITDA','EV/Sales','P/E','DVD Yield','FCF Yield']
    pe = pd.Series()
    ev_to_ebitda = pd.Series()
    ev_to_sales = pd.Series()
    dvd_yield = pd.Series()
    px = bbgclient.bbgclient.get_timeseries(ticker,'PX_LAST',start_date_yyyymmdd,unaffected_date_yyyymmdd,api_host=api_host)
    if 'EV/EBITDA' in multiples_to_query:
        ev_to_ebitda = bbgclient.bbgclient.get_timeseries(ticker,'BEST_CUR_EV_TO_EBITDA',start_date_yyyymmdd,unaffected_date_yyyymmdd,{'BEST_FPERIOD_OVERRIDE':fperiod},api_host)
    if 'EV/Sales' in multiples_to_query:
        ev_to_sales = bbgclient.bbgclient.get_timeseries(ticker,'BEST_CURRENT_EV_BEST_SALES',start_date_yyyymmdd,unaffected_date_yyyymmdd,{'BEST_FPERIOD_OVERRIDE':fperiod},api_host)
    if 'P/E' in multiples_to_query:
        pe = bbgclient.bbgclient.get_timeseries(ticker,'BEST_PE_RATIO',start_date_yyyymmdd,unaffected_date_yyyymmdd,{'BEST_FPERIOD_OVERRIDE':fperiod},api_host)
    if 'DVD Yield' in multiples_to_query:
        dvd_yield = bbgclient.bbgclient.get_timeseries(ticker,'DIVIDEND_INDICATED_YIELD',start_date_yyyymmdd,unaffected_date_yyyymmdd,api_host=api_host)

    df = px.reset_index().rename(columns={'index':'Date',0:'PX'})
    if 'FCF Yield' in multiples_to_query:
        #### FCF-related mnemonics
        ebitda = bbgclient.bbgclient.get_timeseries(ticker,'BEST_EBITDA',start_date_yyyymmdd,unaffected_date_yyyymmdd,{'BEST_FPERIOD_OVERRIDE':fperiod},api_host).reset_index().rename(columns={'index':'Date',0:'EBITDA'})
        opp = bbgclient.bbgclient.get_timeseries(ticker,'BEST_OPP',start_date_yyyymmdd,unaffected_date_yyyymmdd,{'BEST_FPERIOD_OVERRIDE':fperiod},api_host).reset_index().rename(columns={'index':'Date',0:'OPP'})
        capex = bbgclient.bbgclient.get_timeseries(ticker,'BEST_CAPEX',start_date_yyyymmdd,unaffected_date_yyyymmdd,{'BEST_FPERIOD_OVERRIDE':fperiod},api_host).reset_index().rename(columns={'index':'Date',0:'CAPEX'})
        eqy_sh_out = bbgclient.bbgclient.get_timeseries(ticker,'EQY_SH_OUT',start_date_yyyymmdd,unaffected_date_yyyymmdd,api_host=api_host).reset_index().rename(columns={'index':'Date',0:'EQY_SH_OUT'})
        ni = bbgclient.bbgclient.get_timeseries(ticker,'BEST_NET_INCOME',start_date_yyyymmdd,unaffected_date_yyyymmdd,{'BEST_FPERIOD_OVERRIDE':fperiod},api_host).reset_index().rename(columns={'index':'Date',0:'NI'})
        #eps = bbgclient.bbgclient.get_timeseries(ticker,'BEST_EPS',start_date_yyyymmdd,unaffected_date_yyyymmdd,{'BEST_FPERIOD_OVERRIDE':fperiod},api_host).reset_index().rename(columns={'index':'Date',0:'EPS'})
        ####
        fcf = pd.merge(px.reset_index().rename(columns={'index':'Date',0:'PX'}),ebitda, how='left', on=['Date']).ffill().bfill()
        fcf = pd.merge(fcf,opp, how='left', on=['Date']).ffill().bfill()
        fcf = pd.merge(fcf,capex, how='left', on=['Date']).ffill().bfill()
        fcf = pd.merge(fcf,eqy_sh_out, how='left', on=['Date']).ffill().bfill()
        fcf = pd.merge(fcf,ni, how='left', on=['Date']).ffill().bfill()
        #fcf = pd.merge(fcf,eps, how='left', on=['Date']).ffill().bfill()
        #fcf['FCF'] = (fcf['NI'] + fcf['EBITDA'] - fcf['OPP'] + fcf['CAPEX'])/(fcf['NI']/fcf['EPS'])
        fcf['FCF'] = (fcf['NI'] + fcf['EBITDA'] - fcf['OPP'] + fcf['CAPEX'])/fcf['EQY_SH_OUT']
        fcf['FCF Yield'] = fcf['FCF']/fcf['PX']
        df = pd.merge(df, fcf[['Date','FCF Yield']], how='left',on='Date').ffill().bfill()

    df = pd.merge(df,pe.reset_index().rename(columns={'index':'Date',0:'P/E'}), how='left', on=['Date']).ffill().bfill()
    df = pd.merge(df, ev_to_ebitda.reset_index().rename(columns={'index':'Date',0:'EV/EBITDA'}), how='left',on='Date').ffill().bfill()
    df = pd.merge(df, ev_to_sales.reset_index().rename(columns={'index':'Date',0:'EV/Sales'}), how='left',on='Date').ffill().bfill()
    df = pd.merge(df, dvd_yield.reset_index().rename(columns={'index':'Date',0:'DVD Yield'}), how='left',on='Date').ffill().bfill()

    # Date     PX     P/E  EV/EBITDA  EV/Sales  DVD Yield  FCF Yield
    return df

In [6]:
def compute_implied_price_from_multiple(metric_name, multiple, mult_underlying_df):
    try:
        if metric_name == 'EV/EBITDA':
            ebitda = float(mult_underlying_df['BEST_EBITDA'].iloc[0])
            eqy_sh_out = float(mult_underlying_df['EQY_SH_OUT'].iloc[0])
            ev_component = float(mult_underlying_df['CUR_EV_COMPONENT'].iloc[0])
            # net_debt =  float(mult_underlying_df['BEST_NET_DEBT'][0])
            # minority = float(mult_underlying_df['MINORITY_NONCONTROLLING_INTEREST'][0])
            # pfd_eqy = float(mult_underlying_df['BS_PFD_EQY'][0])
            return ((multiple*ebitda)-ev_component)/eqy_sh_out

        if metric_name == 'EV/Sales':
            sales = float(mult_underlying_df['BEST_SALES'].iloc[0])
            eqy_sh_out = float(mult_underlying_df['EQY_SH_OUT'].iloc[0])
            ev_component = float(mult_underlying_df['CUR_EV_COMPONENT'].iloc[0])
            #net_debt =  float(mult_underlying_df['BEST_NET_DEBT'].iloc[0])
            #pfd_eqy = float(mult_underlying_df['BS_PFD_EQY'].iloc[0])
            #minority = float(mult_underlying_df['MINORITY_NONCONTROLLING_INTEREST'].iloc[0])
            return ((multiple*sales)-ev_component)/eqy_sh_out

        if metric_name == 'P/E':
            eps = float(mult_underlying_df['BEST_EPS'].iloc[0])
            return eps*multiple

        if metric_name == 'DVD Yield':
            curr_dvd_yield = float(mult_underlying_df['DIVIDEND_INDICATED_YIELD'].iloc[0])
            curr_px = float(mult_underlying_df['PX'].iloc[0])
            curr_dvd = curr_dvd_yield*curr_px
            implied_px = curr_dvd/multiple
            return implied_px

        if metric_name == 'FCF Yield':
            ni = float(mult_underlying_df['BEST_NET_INCOME'].iloc[0])
            ebitda = float(mult_underlying_df['BEST_EBITDA'].iloc[0])
            opp = float(mult_underlying_df['BEST_OPP'].iloc[0])
            capex = float(mult_underlying_df['BEST_CAPEX'].iloc[0])
            #eps = float(mult_underlying_df['BEST_EPS'].iloc[0])
            eqy_sh_out = float(mult_underlying_df['EQY_SH_OUT'].iloc[0])
            fcf = (ni + ebitda - opp + capex)/eqy_sh_out
            # fcf = (ni + ebitda - opp + capex)/(ni/eps)
            implied_px = fcf/multiple
            return implied_px

    except Exception as e:
        print('failed calculating implied price from multiple: ' + str(metric_name) + ' ' + str(e.args))
        #dbutils.Wic.log('ESS PREMIUM ANALYSIS','failed calculating implied price from multiple: ' + str(e.message))
        return None

In [7]:
def bloomberg_peers(alpha):
    potential_peers = []
    peer = bbgclient.bbgclient.get_secid2field([alpha], "tickers", ['BLOOMBERG_PEERS'], req_type = "refdata")
    for i in range(0, len(peer[alpha]["BLOOMBERG_PEERS"])):
        potential_peers.append(peer[alpha]["BLOOMBERG_PEERS"][i]["Peer Ticker"] + " EQUITY")
    
    return(potential_peers)

In [6]:
def run_regression_optimal_peers(alpha_ticker, unaffect_dt, lookback_period = 120, fperiod = "1BF"):
    metrics = ['P/E', "EV/EBITDA", "EV/Sales", 'DVD Yield', 'FCF Yield']
    api_host = bbgclient.bbgclient.get_next_available_host()
    slicer = dfutils.df_slicer()
    unaff_dt = datetime.datetime.strptime(unaffect_dt, '%Y-%m-%d')
    
    peer_ticker_list = bloomberg_peers(alpha_ticker)
    
    alpha_historical_mult_df = multiples_df(alpha_ticker, slicer.prev_n_business_days(lookback_period,unaff_dt).strftime('%Y%m%d'), unaff_dt.strftime('%Y%m%d'), api_host, fperiod) # ['CID','Date','Ticker','PX','P/E','EV/EBITDA','EV/Sales','DVD yield','FCF yield']
    peer2historical_mult_df = {p:multiples_df(p,slicer.prev_n_business_days(lookback_period,unaff_dt).strftime('%Y%m%d'), unaff_dt.strftime('%Y%m%d'), api_host, fperiod, multiples_to_query=metrics) for p in peer_ticker_list}
    ticker2short_ticker = {p:p.split(' ')[0] for p in peer_ticker_list+[alpha_ticker]}
    
    #peer2ptd_multiple = {p:multiples_df(p,slicer.prev_n_business_days(100,price_tgt_dt).strftime('%Y%m%d'),price_tgt_dt.strftime('%Y%m%d'),api_host,fperiod='1BF',multiples_to_query=metrics) for p in peer_ticker_list}
    #peer2now_multiple = {p:multiples_df(p,slicer.prev_n_business_days(100,as_of_dt).strftime('%Y%m%d'),as_of_dt.strftime('%Y%m%d'),api_host,fperiod='1BF',multiples_to_query=metrics) for p in peer_ticker_list}
    
    #Runs the regression on all metrics and find the optimal peer group per metric which is then uses to calculate the downside
    rows = {}
    metric2peer2coeff = {m:{} for m in metrics}
    peers_d = {}
    for metric in metrics:
        if len(alpha_historical_mult_df[~pd.isnull(alpha_historical_mult_df[metric])])>0:
            m_df = alpha_historical_mult_df[['Date',metric]].rename(columns={metric:ticker2short_ticker[alpha_ticker]})
            for p in peer2historical_mult_df:
                m_df = pd.merge(m_df,peer2historical_mult_df[p][['Date',metric]],how='left',on='Date').rename(columns={metric:ticker2short_ticker[p]})
            peer_tickers = [p for p in peer_ticker_list if len(m_df[~pd.isnull(m_df[p.split(' ')[0]])])>0] # remove peers with all nulls
            #peer_tickers = [p for p in peer_tickers if ~peer2ptd_multiple[p][metric].isnull().all() and ~peer2now_multiple[p][metric].isnull().all()]
            m_ols_df = m_df[[alpha_ticker.split(' ')[0]]+[t.split(' ')[0] for t in peer_tickers]]
            formula = alpha_ticker.split(' ')[0] + '~.' #+ " + ".join([t.split(' ')[0] for t in peer_ticker_list])
            r_df = pandas2ri.py2ri(m_ols_df)
            r_df = stats.na_omit(r_df)
            model = stats.lm(formula, data = r_df)
            optimal_model = stats.step(model, direction = "backward", trace = False)
            summary_results = base.summary(optimal_model)
            summary_results2 = pandas2ri.ri2py(summary_results)
            optimal_peers = base.attr(optimal_model.rx2('terms'), "term.labels")
            optimal_peers2 = []
            for i in optimal_peers:
                for k in peer_tickers:
                    if k.startswith(i + ' '):
                        optimal_peers2.append(k)
                    elif i.startswith('X') and k.startswith(i[1:] + ' '):
                        optimal_peers2.append(k)
                    elif '.' in i:
                        i = i.replace('.', '/')
                        if k.startswith(i + ' '):
                            optimal_peers2.append(k)
                    #Add the "__ EQUITY" phrase back into the optimal peer list for Bloomberg search
            peers_d[metric] = optimal_peers2
            optimal_model_coeff = optimal_model.rx2('coefficients')
            optimal_model_coeff_df = pd.DataFrame(columns=['Peer','Coefficient'],data=[(optimal_peers2[count],optimal_model_coeff[count + 1]) for count in range(0,len(optimal_model_coeff.names)-1)])
            optimal_model_coeff_df = optimal_model_coeff_df.append({'Peer': 'Intercept', 'Coefficient': optimal_model_coeff[0]}, ignore_index = True)
            peer2coeff = {p:optimal_model_coeff.rx2(p)[0] for p in optimal_model_coeff.names}
            del peer2coeff["(Intercept)"]
            count = 0
            for k in optimal_peers:
                peer2coeff[optimal_peers2[count]] = peer2coeff.pop(k)
                count = count + 1
            peer2coeff["(Intercept)"] = optimal_model_coeff[0]
            metric2peer2coeff[metric] = peer2coeff
            #rows.append([metric, summary_results2, optimal_model_coeff_df])
            #rows[metric] = optimal_model_coeff_df
        else:
            continue
            
    try:
        peer_set = {k:set(v) for k, v in peers_d.items()}
        overlap_peers = set.intersection(*peer_set.values()) #Select overlap group from the optimal regression peers above
    
        #Regression of the overlap group which is then used to calculate downside based on all metrics
        if overlap_peers != set():
            rows2 = {}
            metric2peer2coeff2 = {m:{} for m in metrics}
            for metric in metrics:
                if len(alpha_historical_mult_df[~pd.isnull(alpha_historical_mult_df[metric])])>0:
                    m_df2 = alpha_historical_mult_df[['Date',metric]].rename(columns={metric:ticker2short_ticker[alpha_ticker]})
                    for p in overlap_peers:
                        m_df2 = pd.merge(m_df2,peer2historical_mult_df[p][['Date',metric]],how='left',on='Date').rename(columns={metric:ticker2short_ticker[p]})
                    overlap_peers = [p for p in overlap_peers if len(m_df[~pd.isnull(m_df[p.split(' ')[0]])])>0] # remove peers with all nulls
                    m_ols_df2= m_df2[[alpha_ticker.split(' ')[0]]+[t.split(' ')[0] for t in overlap_peers]]
                    #regress a vs. p1,p2,...,pn
                    formula = alpha_ticker.split(' ')[0] + ' ~. ' #+ " + ".join([t.split(' ')[0] for t in overlap_peers])
                    r_df2 = pandas2ri.py2ri(m_ols_df2)
                    r_df2 = stats.na_omit(r_df2)
                    model2 = stats.lm(formula, data = r_df2)
                    #ols_result = sm.ols(formula=formula, data=m_ols_df).fit()
                    summary = base.summary(model2)
                    summary = pandas2ri.ri2py(summary)
                    model_coeff = model2.rx2('coefficients')
                    model_terms = base.attr(model2.rx2('terms'), "term.labels")
                    overlap_peers2 = []
                    for i in model_terms:
                        for k in overlap_peers:
                            if k.startswith(i + ' '):
                                overlap_peers2.append(k)
                            elif i.startswith('X') and k.startswith(i[1:] + ' '):
                                overlap_peers2.append(k)
                            elif '.' in i:
                                i = i.replace('.', '/')
                                if k.startswith(i + ' '):
                                    overlap_peers2.append(k)
                            #Add the "__ EQUITY" phrase back into the optimal peer list for Bloomberg search
                    peer2coeff2 = {}
                    for p in range(0,len(overlap_peers)):
                        peer2coeff2[overlap_peers2[p]] = model_coeff[p+1]
                    peer2coeff2['Intercept'] = model_coeff[0]
                    metric2peer2coeff2[metric] = peer2coeff2
                    #rows2.append([metric,summary]+[peer2coeff2[p] for p in overlap_peers2]+[peer2coeff2['Intercept']])
                else:
                    continue
                    
            return(metric2peer2coeff, metric2peer2coeff2)
    except:
        print("An exception occurred. There may not be an overlap group.")
            
    return(metric2peer2coeff)

In [7]:
metrics = ['P/E', "EV/EBITDA", "EV/Sales", 'DVD Yield', 'FCF Yield']
results = run_regression_optimal_peers('NXST US EQUITY', '2018-11-30', lookback_period = 120)

In [8]:
results
#if len(results) == 2:
   #optimal_results = results[0]
    #overlap_results = results[1]
#else:
    #optimal_results = results
        
#peers = []
#for metric in metrics:
    #results = optimal_results[metric]
    #for x in optimal_results[metric]:
        #if x not in peers:
            #peers.append(x)

({'P/E': {'TRCO US EQUITY': -0.16555075615206588,
   'TGNA US EQUITY': 1.0910868440149244,
   'SSP US EQUITY': -0.0226993158455548,
   'GTN US EQUITY': 0.11869245749932918,
   'WWE US EQUITY': 0.024632630010377517,
   'ETM US EQUITY': 0.3179984328955272,
   'LSXMA US EQUITY': 0.09633740369601836,
   '(Intercept)': -1.1173715762444674},
  'EV/EBITDA': {'SBGI US EQUITY': 0.11968048351637216,
   'TRCO US EQUITY': -0.10325875783451555,
   'TGNA US EQUITY': 0.8000987101564525,
   'SSP US EQUITY': -0.04796910947696571,
   'GTN US EQUITY': 0.09954399656906081,
   'WWE US EQUITY': 0.021559923238494243,
   'SIRI US EQUITY': -0.029570272159093507,
   'ETM US EQUITY': 0.0688011841568601,
   'LSXMA US EQUITY': 0.0769368869488909,
   '(Intercept)': 0.26808235010797393},
  'EV/Sales': {'SBGI US EQUITY': 0.15314277216868882,
   'TRCO US EQUITY': -0.19758962371026992,
   'TGNA US EQUITY': 0.7189999103426479,
   'GTN US EQUITY': 0.1998898094647301,
   'WWE US EQUITY': 0.03380079034763303,
   'SIRI US E

In [9]:
#peers.remove('(Intercept)')
#peers

In [54]:
#list(optimal_results['P/E'].keys()).remove('(Intercept)')

In [22]:
def premium_analysis_df_OLS(alpha_ticker, regression_results, analyst_upside, tgt_dt, api_host, fperiod = "1BF"):
    metrics = ['P/E', "EV/EBITDA", "EV/Sales", 'DVD Yield', 'FCF Yield']
    slicer = dfutils.df_slicer()
    price_tgt_dt = datetime.datetime.strptime(tgt_dt, '%Y-%m-%d')
    as_of_dt = datetime.datetime.today()
    
    if len(regression_results) == 2:
        optimal_results = regression_results[0]
        overlap_results = regression_results[1]
    else:
        optimal_results = regression_results
        
    peers = []
    for metric in metrics:
        results = optimal_results[metric]
        for x in optimal_results[metric]:
            if x not in peers:
                peers.append(x)
    peers.remove('(Intercept)')
    
    peer2ptd_multiple = {p:multiples_df(p,slicer.prev_n_business_days(100,price_tgt_dt).strftime('%Y%m%d'),price_tgt_dt.strftime('%Y%m%d'),api_host,fperiod='1BF',multiples_to_query=metrics) for p in peers}
    peer2now_multiple = {p:multiples_df(p,slicer.prev_n_business_days(100,as_of_dt).strftime('%Y%m%d'),as_of_dt.strftime('%Y%m%d'),api_host,fperiod='1BF',multiples_to_query=metrics) for p in peers}
    alpha_balance_sheet_df_ptd = multiple_underlying_df(alpha_ticker,slicer.prev_n_business_days(100,price_tgt_dt).strftime('%Y%m%d'),price_tgt_dt.strftime('%Y%m%d'),api_host,fperiod='1BF')
    alpha_balance_sheet_df_now = multiple_underlying_df(alpha_ticker,slicer.prev_n_business_days(100,as_of_dt).strftime('%Y%m%d'),as_of_dt.strftime('%Y%m%d'),api_host,fperiod='1BF')
    
    col_metrics = []
    peers_d = {}
    for metric in metrics:
        if metric in optimal_results.keys():
            peers_d[metric] = list(optimal_results[metric].keys())
            col_metrics.append(metric)
    for metric in metrics:
        del peers_d[metric][-1]
        
    df = pd.DataFrame(columns=['Metric'] ,data= col_metrics)
    df['Peers Multiples DataFrame @ Price Target Date'] = df['Metric'].apply(lambda m: pd.DataFrame(columns=['Peer','Multiple'],data=[(p,peer2ptd_multiple[p][m].fillna(0).iloc[-1]) for p in peers_d[m]]))
    df['Alpha Implied Multiple @ Price Target Date'] = df['Metric'].apply(lambda m: sum([optimal_results[m][p]*peer2ptd_multiple[p][m].fillna(0).iloc[-1] for p in peers_d[m]])+optimal_results[m]['(Intercept)'])
    df['Alpha Balance Sheet DataFrame @ Price Target Date'] =  [alpha_balance_sheet_df_ptd]*len(df)
    df['Alpha Unaffected PX @ Price Target Date'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_ptd) for (m,mult) in zip(df['Metric'],df['Alpha Implied Multiple @ Price Target Date'])]

    df['Peers Multiples DataFrame @ Now'] = df['Metric'].apply(lambda m: pd.DataFrame(columns=['Peer','Multiple'],data=[(p,peer2now_multiple[p][m].fillna(0).iloc[-1]) for p in peers_d[m]]))
    df['Alpha Implied Multiple @ Now'] = df['Metric'].apply(lambda m: sum([optimal_results[m][p]*peer2now_multiple[p][m].fillna(0).iloc[-1] for p in peers_d[m]])+optimal_results[m]['(Intercept)'])
    df['Alpha Balance Sheet DataFrame @ Now'] =  [alpha_balance_sheet_df_now]*len(df)
    df['Alpha Unaffected PX @ Now'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_now) for (m,mult) in zip(df['Metric'],df['Alpha Implied Multiple @ Now'])]

    df['Alpha Upside (analyst)'] = analyst_upside
    df['Premium'] = (df['Alpha Upside (analyst)'].astype(float)-df['Alpha Unaffected PX @ Price Target Date'].astype(float)).fillna(0).apply(lambda x: max(x,0))

    df['Alpha Downside (Adj,weighted)'] = df['Alpha Unaffected PX @ Now'].astype(float)
    df['Alpha Upside (Adj,weighted)'] = df['Alpha Unaffected PX @ Now'].astype(float)+df['Premium'].astype(float)
    print(df['Alpha Balance Sheet DataFrame @ Now'][0])
    
    if len(regression_results) == 2:
        for metric in metrics:
            if metric in overlap_results.keys():
                overlap_peers = list(overlap_results[metric].keys())
                break
    
        overlap_peers.remove('Intercept')

        df2 = pd.DataFrame(columns = ['Metric'], data = col_metrics)
        df2['Peers Multiples DataFrame @ Price Target Date'] = df2['Metric'].apply(lambda m: pd.DataFrame(columns=['Peer','Multiple'],data=[(p,peer2ptd_multiple[p][m].fillna(0).iloc[-1]) for p in overlap_peers]))
        df2['Alpha Implied Multiple @ Price Target Date'] = df2['Metric'].apply(lambda m: sum([overlap_results[m][p]*peer2ptd_multiple[p][m].fillna(0).iloc[-1] for p in overlap_peers]) + overlap_results[m]['Intercept'])
        df2['Alpha Balance Sheet DataFrame @ Price Target Date'] =  [alpha_balance_sheet_df_ptd]*len(df2)
        df2['Alpha Unaffected PX @ Price Target Date'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_ptd) for (m,mult) in zip(df2['Metric'],df2['Alpha Implied Multiple @ Price Target Date'])]

        df2['Peers Multiples DataFrame @ Now'] = df2['Metric'].apply(lambda m: pd.DataFrame(columns=['Peer','Multiple'],data=[(p,peer2now_multiple[p][m].fillna(0).iloc[-1]) for p in overlap_peers]))
        df2['Alpha Implied Multiple @ Now'] = df2['Metric'].apply(lambda m: sum([overlap_results[m][p]*peer2now_multiple[p][m].fillna(0).iloc[-1] for p in overlap_peers]) + overlap_results[m]['Intercept'])
        df2['Alpha Balance Sheet DataFrame @ Now'] =  [alpha_balance_sheet_df_now]*len(df2)
        df2['Alpha Unaffected PX @ Now'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_now) for (m,mult) in zip(df2['Metric'],df2['Alpha Implied Multiple @ Now'])]

        df2['Alpha Upside (analyst)'] = analyst_upside
        df2['Premium'] = (df2['Alpha Upside (analyst)'].astype(float)-df2['Alpha Unaffected PX @ Price Target Date'].astype(float)).fillna(0).apply(lambda x: max(x,0))

        df2['Alpha Downside (Adj)'] = df2['Alpha Unaffected PX @ Now'].astype(float)
        df2['Alpha Upside (Adj)'] = df2['Alpha Unaffected PX @ Now'].astype(float)+df2['Premium'].astype(float)

        optimal_peer_results = {df['Metric'][i]: {'Alpha Downside': df['Alpha Downside (Adj,weighted)'][i], 'Alpha Upside': df['Alpha Upside (Adj,weighted)'][i]} for i in range(0,len(df))}
                
        overlap_peer_results = {df2['Metric'][i]: {'Alpha Downside': df2['Alpha Downside (Adj)'][i], 'Alpha Upside': df2['Alpha Upside (Adj)'][i]} for i in range(0,len(df2))}
                
        return { 'optimal_peer_results': optimal_peer_results,
                   'overlap_peer_results': overlap_peer_results
                }

    optimal_peer_results = {df['Metric'][i]: {'Alpha Downside': df['Alpha Downside (Adj,weighted)'][i], 'Alpha Upside': df['Alpha Upside (Adj,weighted)'][i]} for i in range(0,len(df))}
        
    return {
        'optimal_peer_results': optimal_peer_results
    }

In [23]:
metrics = ['P/E', "EV/EBITDA", "EV/Sales", 'DVD Yield', 'FCF Yield']
alpha_ticker = 'NXST US EQUITY'
analyst_upside = 97.50
analyst_downside = 61
analyst_pt_wic = 83
lookback = 120
unaffected_date = '2018-11-30'
target_date = "2019-01-08"
api_host = bbgclient.bbgclient.get_next_available_host()
fperiod = "1BF"

In [24]:
OlS_results = premium_analysis_df_OLS(alpha_ticker, results, analyst_upside, target_date, api_host, fperiod = '1BF')

        Date     PX  CUR_MKT_CAP  EQY_SH_OUT  BEST_EBITDA  BEST_SALES  \
0 2019-02-21  90.84    4143.4335      45.612      912.029    2911.988   

   BEST_EPS  DIVIDEND_INDICATED_YIELD  BEST_OPP  BEST_NET_INCOME  BEST_CAPEX  \
0     7.204                    1.9815   271.167          342.473       -24.6   

   CUR_EV_COMPONENT  
0               NaN  


In [12]:
OlS_results

{'optimal_peer_results': {'P/E': {'Alpha Downside': 70.51402043894409,
   'Alpha Upside': 104.49327626565227},
  'EV/EBITDA': {'Alpha Downside': nan, 'Alpha Upside': nan},
  'EV/Sales': {'Alpha Downside': nan, 'Alpha Upside': nan},
  'DVD Yield': {'Alpha Downside': 95.90814422971006,
   'Alpha Upside': 119.1347197281244},
  'FCF Yield': {'Alpha Downside': 144.8159370720732,
   'Alpha Upside': 144.8159370720732}},
 'overlap_peer_results': {'P/E': {'Alpha Downside': 67.79420582573738,
   'Alpha Upside': 106.77364934072997},
  'EV/EBITDA': {'Alpha Downside': nan, 'Alpha Upside': nan},
  'EV/Sales': {'Alpha Downside': nan, 'Alpha Upside': nan},
  'DVD Yield': {'Alpha Downside': 95.08349692799186,
   'Alpha Upside': 118.47875412918901},
  'FCF Yield': {'Alpha Downside': 148.86027492414001,
   'Alpha Upside': 148.86027492414001}}}

In [10]:
OlS_results['overlap_peers_df']

,Metric,OLS HTML,TGNA US EQUITY,TRCO US EQUITY,ETM US EQUITY,Intercept,Peers Multiples DataFrame @ Price Target Date,Alpha Implied Multiple @ Price Target Date,Alpha Balance Sheet DataFrame @ Price Target Date,Alpha Unaffected PX @ Price Target Date,Peers Multiples DataFrame @ Now,Alpha Implied Multiple @ Now,Alpha Balance Sheet DataFrame @ Now,Alpha Unaffected PX @ Now,Alpha Upside (analyst),Premium,Alpha Downside (Adj),Alpha Upside (Adj)
0,P/E,"[[function (formula, data, subset, weights, na...",1.141343,-0.308632,0.508519,3.452756,Peer Multiple 0 TGNA US EQUITY ...,8.600905,Date PX CUR_MKT_CAP EQY_SH_OUT ...,58.520556,Peer Multiple 0 TGNA US EQUITY ...,8.666008,Date PX CUR_MKT_CAP EQY_SH_OUT ...,60.384746,97.5,38.979444,60.384746,99.364190
1,EV/EBITDA,"[[function (formula, data, subset, weights, na...",0.684908,-0.063665,0.105480,2.732215,Peer Multiple 0 TGNA US EQUITY ...,8.190933,Date PX CUR_MKT_CAP EQY_SH_OUT ...,70.143993,Peer Multiple 0 TGNA US EQUITY ...,8.141707,Date PX CUR_MKT_CAP EQY_SH_OUT ...,71.734658,97.5,27.356007,71.734658,99.090665
2,EV/Sales,"[[function (formula, data, subset, weights, na...",1.187691,-0.233131,0.801285,-1.127999,Peer Multiple 0 TGNA US EQUITY ...,2.582720,Date PX CUR_MKT_CAP EQY_SH_OUT ...,72.842653,Peer Multiple 0 TGNA US EQUITY ...,2.558574,Date PX CUR_MKT_CAP EQY_SH_OUT ...,74.427333,97.5,24.657347,74.427333,99.084681
3,DVD Yield,"[[function (formula, data, subset, weights, na...",0.471433,-0.077793,0.119566,0.416852,Peer Multiple 0 TGNA US EQUITY ...,2.024175,Date PX CUR_MKT_CAP EQY_SH_OUT ...,74.104743,Peer Multiple 0 TGNA US EQUITY ...,2.010023,Date PX CUR_MKT_CAP EQY_SH_OUT ...,89.551327,97.5,23.395257,89.551327,112.946585
4,FCF Yield,"[[function (formula, data, subset, weights, na...",0.636926,-0.022187,0.126497,0.035923,Peer Multiple 0 TGNA US EQUITY ...,0.145467,Date PX CUR_MKT_CAP EQY_SH_OUT ...,139.295919,Peer Multiple 0 TGNA US EQUITY ...,0.146916,Date PX CUR_MKT_CAP EQY_SH_OUT ...,139.856708,97.5,0.000000,139.856708,139.856708


In [120]:
def premium_analysis_df_OLS_quick(dataframes, regression_results, alpha_ticker, analyst_upside, tgt_dt, api_host, fperiod = "1BF"):
    metrics = ['P/E', "EV/EBITDA", "EV/Sales", 'DVD Yield', 'FCF Yield']
    slicer = dfutils.df_slicer()
    price_tgt_dt = datetime.datetime.strptime(tgt_dt, '%Y-%m-%d')
    as_of_dt = datetime.datetime.today()
    
    if len(regression_results) == 2:
        optimal_results = regression_results[0]
        overlap_results = regression_results[1]
    else:
        optimal_results = regression_results
        
    peers = []
    for metric in metrics:
        for x in optimal_results[metric]:
            if x not in peers:
                peers.append(x)
    peers.remove('(Intercept)')
    
    peer2ptd_multiple = dataframes['peer2ptd_multiple']
    peer2now_multiple = dataframes['peer2now_multiple']
    alpha_balance_sheet_df_ptd = multiple_underlying_df(alpha_ticker,slicer.prev_n_business_days(100,price_tgt_dt).strftime('%Y%m%d'),price_tgt_dt.strftime('%Y%m%d'),api_host,fperiod='1BF')
    alpha_balance_sheet_df_now = multiple_underlying_df(alpha_ticker,slicer.prev_n_business_days(100,as_of_dt).strftime('%Y%m%d'),as_of_dt.strftime('%Y%m%d'),api_host,fperiod='1BF')

    col_metrics = []
    peers_d = {}
    for metric in metrics:
        if metric in optimal_results.keys():
            peers_d[metric] = list(optimal_results[metric].keys())
            col_metrics.append(metric)
    for metric in metrics:
        del peers_d[metric][-1]
    
    df = pd.DataFrame(columns=['Metric'], data = col_metrics)
    df['Peers Multiples DataFrame @ Price Target Date'] = df['Metric'].apply(lambda m: pd.DataFrame(columns=['Peer','Multiple'],data=[(p,peer2ptd_multiple[p][m].fillna(0).iloc[-1]) for p in peers_d[m]]))
    df['Alpha Implied Multiple @ Price Target Date'] = df['Metric'].apply(lambda m: sum([optimal_results[m][p]*peer2ptd_multiple[p][m].fillna(0).iloc[-1] for p in peers_d[m]])+ optimal_results[m]['(Intercept)'])
    df['Alpha Balance Sheet DataFrame @ Price Target Date'] =  [alpha_balance_sheet_df_ptd]*len(df)
    df['Alpha Unaffected PX @ Price Target Date'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_ptd) for (m,mult) in zip(df['Metric'],df['Alpha Implied Multiple @ Price Target Date'])]

    df['Peers Multiples DataFrame @ Now'] = df['Metric'].apply(lambda m: pd.DataFrame(columns=['Peer','Multiple'],data=[(p,peer2now_multiple[p][m].fillna(0).iloc[-1]) for p in peers_d[m]]))
    df['Alpha Implied Multiple @ Now'] = df['Metric'].apply(lambda m: sum([optimal_results[m][p]*peer2now_multiple[p][m].fillna(0).iloc[-1] for p in peers_d[m]])+optimal_results[m]['(Intercept)'])
    df['Alpha Balance Sheet DataFrame @ Now'] =  [alpha_balance_sheet_df_now]*len(df)
    df['Alpha Unaffected PX @ Now'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_now) for (m,mult) in zip(df['Metric'],df['Alpha Implied Multiple @ Now'])]

    df['Alpha Upside (analyst)'] = analyst_upside
    df['Premium'] = (df['Alpha Upside (analyst)'].astype(float)-df['Alpha Unaffected PX @ Price Target Date'].astype(float)).fillna(0).apply(lambda x: max(x,0))

    df['Alpha Downside (Adj,weighted)'] = df['Alpha Unaffected PX @ Now'].astype(float)
    df['Alpha Upside (Adj,weighted)'] = df['Alpha Unaffected PX @ Now'].astype(float)+df['Premium'].astype(float)
    

    if len(regression_results) == 2:
        for metric in metrics:
            if metric in overlap_results.keys():
                overlap_peers = list(overlap_results[metric].keys())
                break
    
        overlap_peers.remove('Intercept')

        df2 = pd.DataFrame(columns=['Metric'], data = col_metrics)
        df2['Peers Multiples DataFrame @ Price Target Date'] = df2['Metric'].apply(lambda m: pd.DataFrame(columns = ['Peer','Multiple'],data = [(p,peer2ptd_multiple[p][m].fillna(0).iloc[-1]) for p in overlap_peers]))
        df2['Alpha Implied Multiple @ Price Target Date'] = df2['Metric'].apply(lambda m: sum([overlap_results[m][p]*peer2ptd_multiple[p][m].fillna(0).iloc[-1] for p in overlap_peers]) + overlap_results[m]['Intercept'])
        df2['Alpha Balance Sheet DataFrame @ Price Target Date'] =  [alpha_balance_sheet_df_ptd]*len(df2)
        df2['Alpha Unaffected PX @ Price Target Date'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_ptd) for (m,mult) in zip(df2['Metric'],df2['Alpha Implied Multiple @ Price Target Date'])]

        df2['Peers Multiples DataFrame @ Now'] = df2['Metric'].apply(lambda m: pd.DataFrame(columns = ['Peer','Multiple'],data = [(p,peer2now_multiple[p][m].fillna(0).iloc[-1]) for p in overlap_peers]))
        df2['Alpha Implied Multiple @ Now'] = df2['Metric'].apply(lambda m: sum([overlap_results[m][p]*peer2now_multiple[p][m].fillna(0).iloc[-1] for p in overlap_peers]) + overlap_results[m]['Intercept'])
        df2['Alpha Balance Sheet DataFrame @ Now'] =  [alpha_balance_sheet_df_now]*len(df2)
        df2['Alpha Unaffected PX @ Now'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_now) for (m,mult) in zip(df2['Metric'],df2['Alpha Implied Multiple @ Now'])]

        df2['Alpha Upside (analyst)'] = analyst_upside
        df2['Premium'] = (df2['Alpha Upside (analyst)'].astype(float)-df2['Alpha Unaffected PX @ Price Target Date'].astype(float)).fillna(0).apply(lambda x: max(x,0))

        df2['Alpha Downside (Adj)'] = df2['Alpha Unaffected PX @ Now'].astype(float)
        df2['Alpha Upside (Adj)'] = df2['Alpha Unaffected PX @ Now'].astype(float)+df2['Premium'].astype(float)

        optimal_peer_results = {df['Metric'][i]: {'Alpha Downside': df['Alpha Downside (Adj,weighted)'][i], 'Alpha Upside': df['Alpha Upside (Adj,weighted)'][i]} for i in range(0,len(df))}        
        overlap_peer_results = {df2['Metric'][i]: {'Alpha Downside': df2['Alpha Downside (Adj)'][i], 'Alpha Upside': df2['Alpha Upside (Adj)'][i]} for i in range(0,len(df2))}
                
        return { 'optimal_peer_results': optimal_peer_results,
                'overlap_peer_results': overlap_peer_results
        }
        
    optimal_peer_results = {df['Metric'][i]: {'Alpha Downside': df['Alpha Downside (Adj,weighted)'][i], 'Alpha Upside': df['Alpha Upside (Adj,weighted)'][i]} for i in range(0,len(df))}
        
    return {
        'optimal_peer_results': optimal_peer_results
    }


In [129]:
def premium_analysis_df_OLS2_quick(dataframes, regression_results, alpha_ticker, analyst_upside, analyst_downside, analyst_pt_wic, tgt_dt,  api_host, adjustments_df_now = None, adjustments_df_ptd = None, fperiod = "1BF"):
    metrics = ['P/E', "EV/EBITDA", "EV/Sales", 'DVD Yield', 'FCF Yield']
    slicer = dfutils.df_slicer()
    price_tgt_dt = datetime.datetime.strptime(tgt_dt, '%Y-%m-%d')
    as_of_dt = datetime.datetime.today()
    
    if len(regression_results) == 2:
        optimal_results = regression_results[0]
        overlap_results = regression_results[1]
    else:
        optimal_results = regression_results
        
    peers = []
    for metric in metrics:
        for x in optimal_results[metric]:
            if x not in peers:
                peers.append(x)
    peers.remove('(Intercept)')
    
    peer2ptd_multiple = dataframes['peer2ptd_multiple']
    peer2now_multiple = dataframes['peer2now_multiple']
    alpha_balance_sheet_df_ptd = multiple_underlying_df(alpha_ticker,slicer.prev_n_business_days(100,price_tgt_dt).strftime('%Y%m%d'),price_tgt_dt.strftime('%Y%m%d'),api_host,fperiod='1BF')
    alpha_balance_sheet_df_now = multiple_underlying_df(alpha_ticker,slicer.prev_n_business_days(100,as_of_dt).strftime('%Y%m%d'),as_of_dt.strftime('%Y%m%d'),api_host,fperiod='1BF')
    
    col_metrics = []
    peers_d = {}
    for metric in metrics:
        if metric in optimal_results.keys():
            peers_d[metric] = list(optimal_results[metric].keys())
            col_metrics.append(metric)
    for metric in metrics:
        del peers_d[metric][-1]    
    
    df = pd.DataFrame(columns = ['Metric'], data = col_metrics)
    df['Peers Multiples DataFrame @ Price Target Date'] = df['Metric'].apply(lambda m: pd.DataFrame(columns = ['Peer','Multiple'], data = [(p,peer2ptd_multiple[p][m].fillna(0).iloc[-1]) for p in peers_d[m]]))
    df['Alpha Implied Multiple @ Price Target Date'] = df['Metric'].apply(lambda m: sum([optimal_results[m][p]*peer2ptd_multiple[p][m].fillna(0).iloc[-1] for p in peers_d[m]]) + optimal_results[m]['(Intercept)'])
    
    if adjustments_df_ptd is None:
        df['Alpha Balance Sheet DataFrame @ Price Target Date'] =  [alpha_balance_sheet_df_ptd]*len(df)
        df['Alpha Unaffected PX @ Price Target Date'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_ptd) for (m,mult) in zip(df['Metric'],df['Alpha Implied Multiple @ Price Target Date'])]
    else:
        adjustments_df2 = adjustments_df_ptd.drop(columns = 'Date')
        alpha_balance_sheet_df_ptd_adj = alpha_balance_sheet_df_ptd.add(adjustments_df2, axis = 'columns')
        df['Alpha Balance Sheet DataFrame @ Price Target Date'] =  [alpha_balance_sheet_df_ptd_adj]*len(df)
        df['Alpha Unaffected PX @ Price Target Date'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_ptd_adj) for (m,mult) in zip(df['Metric'],df['Alpha Implied Multiple @ Price Target Date'])]

    df['Peers Multiples DataFrame @ Now'] = df['Metric'].apply(lambda m: pd.DataFrame(columns = ['Peer','Multiple'],data = [(p,peer2now_multiple[p][m].fillna(0).iloc[-1]) for p in peers_d[m]]))
    df['Alpha Implied Multiple @ Now'] = df['Metric'].apply(lambda m: sum([optimal_results[m][p]*peer2now_multiple[p][m].fillna(0).iloc[-1] for p in peers_d[m]]) + optimal_results[m]['(Intercept)'])
    
    if adjustments_df_now is None:
        df['Alpha Balance Sheet DataFrame @ Now'] =  [alpha_balance_sheet_df_now]*len(df)
        df['Alpha Unaffected PX @ Now'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_now) for (m,mult) in zip(df['Metric'],df['Alpha Implied Multiple @ Now'])]
    else:
        adjustments_df1 = adjustments_df_now.drop(columns = 'Date')
        alpha_balance_sheet_df_now_adj = alpha_balance_sheet_df_now.add(adjustments_df1, axis = 'columns')
        df['Alpha Balance Sheet DataFrame @ Now'] =  [alpha_balance_sheet_df_now_adj]*len(df)
        df['Alpha Unaffected PX @ Now'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_now_adj) for (m,mult) in zip(df['Metric'],df['Alpha Implied Multiple @ Now'])]    
        
    df['Alpha Upside (analyst)'] = analyst_upside
    df['Alpha Downside (analyst)'] = analyst_downside
    df['Alpha PT WIC (analyst)'] = analyst_pt_wic
    
    df['Premium Up ($)'] = (df['Alpha Upside (analyst)'].astype(float) - df['Alpha Unaffected PX @ Price Target Date'].astype(float)).fillna(0)
    df['Premium PT WIC ($)'] = (df['Alpha PT WIC (analyst)'].astype(float) - df['Alpha Unaffected PX @ Price Target Date'].astype(float)).fillna(0)
    df['Premium Down ($)'] = (df['Alpha Downside (analyst)'].astype(float) - df['Alpha Unaffected PX @ Price Target Date'].astype(float)).fillna(0)
        
    df['Alpha Downside (Adj)'] = df['Alpha Unaffected PX @ Now'].astype(float) + df['Premium Down ($)'].astype(float)
    df['Alpha PT WIC (Adj)'] = df['Alpha Unaffected PX @ Now'].astype(float) + df['Premium PT WIC ($)'].astype(float)
    df['Alpha Upside (Adj)'] = df['Alpha Unaffected PX @ Now'].astype(float) + df['Premium Up ($)'].astype(float)
    
    if len(regression_results) == 2:
        for metric in metrics:
            if metric in overlap_results.keys():
                overlap_peers = list(overlap_results[metric].keys())
                break
    
        overlap_peers.remove('Intercept')

        df2 = pd.DataFrame(columns = ['Metric'], data = col_metrics)
        df2['Peers Multiples DataFrame @ Price Target Date'] = df2['Metric'].apply(lambda m: pd.DataFrame(columns = ['Peer','Multiple'], data = [(p,peer2ptd_multiple[p][m].fillna(0).iloc[-1]) for p in overlap_peers]))
        df2['Alpha Implied Multiple @ Price Target Date'] = df2['Metric'].apply(lambda m: sum([overlap_results[m][p]*peer2ptd_multiple[p][m].fillna(0).iloc[-1] for p in overlap_peers]) + overlap_results[m]['Intercept'])
        
        if adjustments_df_ptd is None:
            df2['Alpha Balance Sheet DataFrame @ Price Target Date'] =  [alpha_balance_sheet_df_ptd]*len(df2)
            df2['Alpha Unaffected PX @ Price Target Date'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_ptd) for (m,mult) in zip(df2['Metric'],df2['Alpha Implied Multiple @ Price Target Date'])]
        else:
            adjustments_df2 = adjustments_df_ptd.drop(columns = 'Date')
            alpha_balance_sheet_df_ptd_adj = alpha_balance_sheet_df_ptd.add(adjustments_df2, axis = 'columns')
            df2['Alpha Balance Sheet DataFrame @ Price Target Date'] =  [alpha_balance_sheet_df_ptd_adj]*len(df)
            df2['Alpha Unaffected PX @ Price Target Date'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_ptd_adj) for (m,mult) in zip(df['Metric'],df2['Alpha Implied Multiple @ Price Target Date'])]        

        df2['Peers Multiples DataFrame @ Now'] = df2['Metric'].apply(lambda m: pd.DataFrame(columns  =['Peer','Multiple'],data = [(p,peer2now_multiple[p][m].fillna(0).iloc[-1]) for p in overlap_peers]))
        df2['Alpha Implied Multiple @ Now'] = df2['Metric'].apply(lambda m: sum([overlap_results[m][p]*peer2now_multiple[p][m].fillna(0).iloc[-1] for p in overlap_peers]) + overlap_results[m]['Intercept'])
        
        if adjustments_df_now is None:
            df2['Alpha Balance Sheet DataFrame @ Now'] =  [alpha_balance_sheet_df_now]*len(df2)
            df2['Alpha Unaffected PX @ Now'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_now) for (m,mult) in zip(df2['Metric'],df2['Alpha Implied Multiple @ Now'])]
        else:
            adjustments_df1 = adjustments_df_now.drop(columns = 'Date')
            alpha_balance_sheet_df_now_adj = alpha_balance_sheet_df_now.add(adjustments_df1, axis = 'columns')
            df2['Alpha Balance Sheet DataFrame @ Now'] =  [alpha_balance_sheet_df_now_adj]*len(df)
            df2['Alpha Unaffected PX @ Now'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_now_adj) for (m,mult) in zip(df['Metric'],df2['Alpha Implied Multiple @ Now'])]            

        df2['Alpha Upside (analyst)'] = analyst_upside
        df2['Alpha Downside (analyst)'] = analyst_downside
        df2['Alpha PT WIC (analyst)'] = analyst_pt_wic
        
        df2['Premium Up ($)'] = (df2['Alpha Upside (analyst)'].astype(float) - df2['Alpha Unaffected PX @ Price Target Date'].astype(float)).fillna(0)
        df2['Premium PT WIC ($)'] = (df2['Alpha PT WIC (analyst)'].astype(float) - df2['Alpha Unaffected PX @ Price Target Date'].astype(float)).fillna(0)
        df2['Premium Down ($)'] = (df2['Alpha Downside (analyst)'].astype(float)  -df2['Alpha Unaffected PX @ Price Target Date'].astype(float)).fillna(0)

        df2['Alpha Downside (Adj)'] = df2['Alpha Unaffected PX @ Now'].astype(float) + df2['Premium Down ($)'].astype(float)
        df2['Alpha PT WIC (Adj)'] = df2['Alpha Unaffected PX @ Now'].astype(float) + df2['Premium PT WIC ($)'].astype(float)
        df2['Alpha Upside (Adj)'] = df2['Alpha Unaffected PX @ Now'].astype(float ) + df2['Premium Up ($)'].astype(float)

        optimal_peer_results = {df['Metric'][i]: {'Alpha Downside': df['Alpha Downside (Adj)'][i], 'Alpha PT WIC': df['Alpha PT WIC (Adj)'][i], 'Alpha Upside': df['Alpha Upside (Adj)'][i]} for i in range(0,len(df))}
                
        overlap_peer_results = {df2['Metric'][i]: {'Alpha Downside': df2['Alpha Downside (Adj)'][i], 'Alpha PT WIC': df2['Alpha PT WIC (Adj)'][i], 'Alpha Upside': df2['Alpha Upside (Adj)'][i]} for i in range(0,len(df2))}
                
        return { 'optimal_peer_results': optimal_peer_results,
                'overlap_peer_results': overlap_peer_results
                }
        
    optimal_peer_results = {df['Metric'][i]: {'Alpha Downside': df['Alpha Downside (Adj)'][i], 'Alpha PT WIC': df['Alpha PT WIC (Adj)'][i], 'Alpha Upside': df['Alpha Upside (Adj)'][i]} for i in range(0,len(df))}
        
    return {
        'optimal_peer_results': optimal_peer_results
    }
          
    

In [130]:
def run_OLS_regression_analysis(alpha_ticker, tgtDate, analyst_upside, analyst_downside, analyst_pt_wic, regression_results, api_host, adjustments_df_now = None, adjustments_df_ptd = None, f_period = '1BF'):
    metrics = ['P/E', "EV/EBITDA", "EV/Sales", 'DVD Yield', 'FCF Yield']
    slicer = dfutils.df_slicer()
    price_tgt_dt = datetime.datetime.strptime(tgtDate, '%Y-%m-%d')
    as_of_dt = datetime.datetime.today()
    
    if len(regression_results) == 2:
        optimal_results = regression_results[0]
        overlap_results = regression_results[1]
    else:
        optimal_results = regression_results
        
    peers = []
    for metric in metrics:
        results = optimal_results[metric]
        for x in optimal_results[metric]:
            if x not in peers:
                peers.append(x)
    peers.remove('(Intercept)')
    
    peer2ptd_multiple = {p:multiples_df(p,slicer.prev_n_business_days(100,price_tgt_dt).strftime('%Y%m%d'),price_tgt_dt.strftime('%Y%m%d'),api_host,fperiod='1BF',multiples_to_query=metrics) for p in peers}
    peer2now_multiple = {p:multiples_df(p,slicer.prev_n_business_days(100,as_of_dt).strftime('%Y%m%d'),as_of_dt.strftime('%Y%m%d'),api_host,fperiod='1BF',multiples_to_query=metrics) for p in peers}
    
    dataframes = {
        'peer2ptd_multiple': peer2ptd_multiple,
        'peer2now_multiple': peer2now_multiple
    }
    
    original_results = premium_analysis_df_OLS_quick(dataframes, regression_results, alpha_ticker, analyst_upside, tgtDate, api_host, fperiod = "1BF")

    adjusted_results = premium_analysis_df_OLS2_quick(dataframes, regression_results, alpha_ticker, analyst_upside, analyst_downside, analyst_pt_wic, tgtDate, api_host, adjustments_df_now = None, adjustments_df_ptd = None, fperiod = "1BF")
    
    return { 'original_results': original_results, 
            'adjusted_results': adjusted_results
           }

In [131]:
run_OLS_regression_analysis(alpha_ticker, target_date, analyst_upside, analyst_downside, analyst_pt_wic, results, api_host, adjustments_df_now = None, adjustments_df_ptd = None, f_period = '1BF')

{'original_results': {'optimal_peer_results': {'P/E': {'Alpha Downside': 69.80761105250481,
    'Alpha Upside': 103.786866879213},
   'EV/EBITDA': {'Alpha Downside': 71.46308362058154,
    'Alpha Upside': 102.24900069553345},
   'EV/Sales': {'Alpha Downside': 78.4042578735371,
    'Alpha Upside': 98.28895698778209},
   'DVD Yield': {'Alpha Downside': 95.86544653257158,
    'Alpha Upside': 119.09202203098592},
   'FCF Yield': {'Alpha Downside': 144.76374201511692,
    'Alpha Upside': 144.76374201511692}},
  'overlap_peer_results': {'P/E': {'Alpha Downside': 66.52441470019447,
    'Alpha Upside': 105.50385821518705},
   'EV/EBITDA': {'Alpha Downside': 75.78680613702072,
    'Alpha Upside': 103.14281300582888},
   'EV/Sales': {'Alpha Downside': 82.52240405001591,
    'Alpha Upside': 107.17975144833407},
   'DVD Yield': {'Alpha Downside': 94.76338615091005,
    'Alpha Upside': 118.1586433521072},
   'FCF Yield': {'Alpha Downside': 147.66751652453829,
    'Alpha Upside': 147.66751652453829}

In [132]:
def premium_analysis_df_OLS2(alpha_ticker, regression_results, analyst_upside, analyst_downside, analyst_pt_wic, tgt_dt, api_host, adjustments_df_now = None, adjustments_df_ptd = None, fperiod = "1BF"):
    metrics = ['P/E', "EV/EBITDA", "EV/Sales", 'DVD Yield', 'FCF Yield']
    slicer = dfutils.df_slicer()
    price_tgt_dt = datetime.datetime.strptime(tgt_dt, '%Y-%m-%d')
    as_of_dt = datetime.datetime.today()
    
    if len(regression_results) == 2:
        optimal_results = regression_results[0]
        overlap_results = regression_results[1]
    else:
        optimal_results = regression_results
        
    peers = []
    for metric in metrics:
        for x in optimal_results[metric]:
            if x not in peers:
                peers.append(x)
    peers.remove('(Intercept)') 
    
    peer2ptd_multiple = {p:multiples_df(p,slicer.prev_n_business_days(100,price_tgt_dt).strftime('%Y%m%d'),price_tgt_dt.strftime('%Y%m%d'),api_host,fperiod = '1BF',multiples_to_query = metrics) for p in peers}
    peer2now_multiple = {p:multiples_df(p,slicer.prev_n_business_days(100,as_of_dt).strftime('%Y%m%d'),as_of_dt.strftime('%Y%m%d'),api_host,fperiod = '1BF',multiples_to_query = metrics) for p in peers}
    alpha_balance_sheet_df_ptd = multiple_underlying_df(alpha_ticker,slicer.prev_n_business_days(100,price_tgt_dt).strftime('%Y%m%d'),price_tgt_dt.strftime('%Y%m%d'),api_host,fperiod = '1BF')
    alpha_balance_sheet_df_now = multiple_underlying_df(alpha_ticker,slicer.prev_n_business_days(100,as_of_dt).strftime('%Y%m%d'),as_of_dt.strftime('%Y%m%d'),api_host,fperiod = '1BF')
    
    col_metrics = []
    peers_d = {}
    for metric in metrics:
        if metric in optimal_results.keys():
            peers_d[metric] = list(optimal_results[metric].keys())
            col_metrics.append(metric)
    for metric in metrics:
        del peers_d[metric][-1]    
        
    df = pd.DataFrame(columns = ['Metric'], data = col_metrics)
    df['Peers Multiples DataFrame @ Price Target Date'] = df['Metric'].apply(lambda m: pd.DataFrame(columns = ['Peer','Multiple'], data = [(p,peer2ptd_multiple[p][m].fillna(0).iloc[-1]) for p in peers_d[m]]))
    df['Alpha Implied Multiple @ Price Target Date'] = df['Metric'].apply(lambda m: sum([optimal_results[m][p]*peer2ptd_multiple[p][m].fillna(0).iloc[-1] for p in peers_d[m]]) + optimal_results[m]['(Intercept)'])
    
    if adjustments_df_ptd is None:
        df['Alpha Balance Sheet DataFrame @ Price Target Date'] =  [alpha_balance_sheet_df_ptd]*len(df)
        df['Alpha Unaffected PX @ Price Target Date'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_ptd) for (m,mult) in zip(df['Metric'],df['Alpha Implied Multiple @ Price Target Date'])]
    else:
        adjustments_df2 = adjustments_df_ptd.drop(columns = 'Date')
        alpha_balance_sheet_df_ptd_adj = alpha_balance_sheet_df_ptd.add(adjustments_df2, axis = 'columns')
        df['Alpha Balance Sheet DataFrame @ Price Target Date'] =  [alpha_balance_sheet_df_ptd_adj]*len(df)
        df['Alpha Unaffected PX @ Price Target Date'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_ptd_adj) for (m,mult) in zip(df['Metric'],df['Alpha Implied Multiple @ Price Target Date'])]

    df['Peers Multiples DataFrame @ Now'] = df['Metric'].apply(lambda m: pd.DataFrame(columns = ['Peer','Multiple'], data = [(p,peer2now_multiple[p][m].fillna(0).iloc[-1]) for p in peers_d[m]]))
    df['Alpha Implied Multiple @ Now'] = df['Metric'].apply(lambda m: sum([optimal_results[m][p]*peer2now_multiple[p][m].fillna(0).iloc[-1] for p in peers_d[m]]) + optimal_results[m]['(Intercept)'])
    
    if adjustments_df_now is None:
        df['Alpha Balance Sheet DataFrame @ Now'] =  [alpha_balance_sheet_df_now]*len(df)
        df['Alpha Unaffected PX @ Now'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_now) for (m,mult) in zip(df['Metric'],df['Alpha Implied Multiple @ Now'])]
    else:
        adjustments_df1 = adjustments_df_now.drop(columns = 'Date')
        alpha_balance_sheet_df_now_adj = alpha_balance_sheet_df_now.add(adjustments_df1, axis = 'columns')
        df['Alpha Balance Sheet DataFrame @ Now'] =  [alpha_balance_sheet_df_now_adj]*len(df)
        df['Alpha Unaffected PX @ Now'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_now_adj) for (m,mult) in zip(df['Metric'],df['Alpha Implied Multiple @ Now'])]    
        
    df['Alpha Upside (analyst)'] = analyst_upside
    df['Alpha Downside (analyst)'] = analyst_downside
    df['Alpha PT WIC (analyst)'] = analyst_pt_wic
    
    df['Premium Up ($)'] = (df['Alpha Upside (analyst)'].astype(float) - df['Alpha Unaffected PX @ Price Target Date'].astype(float)).fillna(0)
    df['Premium PT WIC ($)'] = (df['Alpha PT WIC (analyst)'].astype(float) - df['Alpha Unaffected PX @ Price Target Date'].astype(float)).fillna(0)
    df['Premium Down ($)'] = (df['Alpha Downside (analyst)'].astype(float) - df['Alpha Unaffected PX @ Price Target Date'].astype(float)).fillna(0)
        
    df['Alpha Downside (Adj)'] = df['Alpha Unaffected PX @ Now'].astype(float) + df['Premium Down ($)'].astype(float)
    df['Alpha PT WIC (Adj)'] = df['Alpha Unaffected PX @ Now'].astype(float) + df['Premium PT WIC ($)'].astype(float)
    df['Alpha Upside (Adj)'] = df['Alpha Unaffected PX @ Now'].astype(float) + df['Premium Up ($)'].astype(float)
    
    if len(regression_results) == 2:
        for metric in metrics:
            if metric in overlap_results.keys():
                overlap_peers = list(overlap_results[metric].keys())
                break
    
        overlap_peers.remove('Intercept')

        df2 = pd.DataFrame(columns = ['Metric'], data = col_metrics)
        df2['Peers Multiples DataFrame @ Price Target Date'] = df2['Metric'].apply(lambda m: pd.DataFrame(columns = ['Peer','Multiple'], data = [(p,peer2ptd_multiple[p][m].fillna(0).iloc[-1]) for p in overlap_peers]))
        df2['Alpha Implied Multiple @ Price Target Date'] = df2['Metric'].apply(lambda m: sum([overlap_results[m][p]*peer2ptd_multiple[p][m].fillna(0).iloc[-1] for p in overlap_peers]) + overlap_results[m]['Intercept'])
        
        if adjustments_df_ptd is None:
            df2['Alpha Balance Sheet DataFrame @ Price Target Date'] =  [alpha_balance_sheet_df_ptd]*len(df2)
            df2['Alpha Unaffected PX @ Price Target Date'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_ptd) for (m,mult) in zip(df2['Metric'],df2['Alpha Implied Multiple @ Price Target Date'])]
        else:
            adjustments_df2 = adjustments_df_ptd.drop(columns = 'Date')
            alpha_balance_sheet_df_ptd_adj = alpha_balance_sheet_df_ptd.add(adjustments_df2, axis = 'columns')
            df2['Alpha Balance Sheet DataFrame @ Price Target Date'] =  [alpha_balance_sheet_df_ptd_adj]*len(df)
            df2['Alpha Unaffected PX @ Price Target Date'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_ptd_adj) for (m,mult) in zip(df['Metric'],df2['Alpha Implied Multiple @ Price Target Date'])]        

        df2['Peers Multiples DataFrame @ Now'] = df2['Metric'].apply(lambda m: pd.DataFrame(columns = ['Peer','Multiple'], data = [(p,peer2now_multiple[p][m].fillna(0).iloc[-1]) for p in overlap_peers]))
        df2['Alpha Implied Multiple @ Now'] = df2['Metric'].apply(lambda m: sum([overlap_results[m][p]*peer2now_multiple[p][m].fillna(0).iloc[-1] for p in overlap_peers]) + overlap_results[m]['Intercept'])
        
        if adjustments_df_now is None:
            df2['Alpha Balance Sheet DataFrame @ Now'] =  [alpha_balance_sheet_df_now]*len(df2)
            df2['Alpha Unaffected PX @ Now'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_now) for (m,mult) in zip(df2['Metric'],df2['Alpha Implied Multiple @ Now'])]
        else:
            adjustments_df1 = adjustments_df_now.drop(columns = 'Date')
            alpha_balance_sheet_df_now_adj = alpha_balance_sheet_df_now.add(adjustments_df1, axis = 'columns')
            df2['Alpha Balance Sheet DataFrame @ Now'] =  [alpha_balance_sheet_df_now_adj]*len(df)
            df2['Alpha Unaffected PX @ Now'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_now_adj) for (m,mult) in zip(df['Metric'],df2['Alpha Implied Multiple @ Now'])]            

        df2['Alpha Upside (analyst)'] = analyst_upside
        df2['Alpha Downside (analyst)'] = analyst_downside
        df2['Alpha PT WIC (analyst)'] = analyst_pt_wic
        
        df2['Premium Up ($)'] = (df2['Alpha Upside (analyst)'].astype(float) - df2['Alpha Unaffected PX @ Price Target Date'].astype(float)).fillna(0)
        df2['Premium PT WIC ($)'] = (df2['Alpha PT WIC (analyst)'].astype(float) - df2['Alpha Unaffected PX @ Price Target Date'].astype(float)).fillna(0)
        df2['Premium Down ($)'] = (df2['Alpha Downside (analyst)'].astype(float) - df2['Alpha Unaffected PX @ Price Target Date'].astype(float)).fillna(0)

        df2['Alpha Downside (Adj)'] = df2['Alpha Unaffected PX @ Now'].astype(float) + df2['Premium Down ($)'].astype(float)
        df2['Alpha PT WIC (Adj)'] = df2['Alpha Unaffected PX @ Now'].astype(float) + df2['Premium PT WIC ($)'].astype(float)
        df2['Alpha Upside (Adj)'] = df2['Alpha Unaffected PX @ Now'].astype(float ) + df2['Premium Up ($)'].astype(float)

        optimal_peer_results = {df['Metric'][i]: {'Alpha Downside': df['Alpha Downside (Adj)'][i], 'Alpha PT WIC': df['Alpha PT WIC (Adj)'][i], 'Alpha Upside': df['Alpha Upside (Adj)'][i]} for i in range(0,len(df))}
                
        overlap_peer_results = {df2['Metric'][i]: {'Alpha Downside': df2['Alpha Downside (Adj)'][i], 'Alpha PT WIC': df2['Alpha PT WIC (Adj)'][i], 'Alpha Upside': df2['Alpha Upside (Adj)'][i]} for i in range(0,len(df2))}
                
        return { 'optimal_peer_results': optimal_peer_results,
                'overlap_peer_results': overlap_peer_results
                }
        
    optimal_peer_results = {df['Metric'][i]: {'Alpha Downside': df['Alpha Downside (Adj)'][i], 'Alpha PT WIC': df['Alpha PT WIC (Adj)'][i], 'Alpha Upside': df['Alpha Upside (Adj)'][i]} for i in range(0,len(df))}
        
    return {
        'optimal_peer_results': optimal_peer_results
    }

In [134]:
premium_analysis_df_OLS2(alpha_ticker, results, analyst_upside, analyst_downside, analyst_pt_wic, target_date, api_host, adjustments_df_now = None, adjustments_df_ptd = None, fperiod = "1BF")

{'optimal_peer_results': {'P/E': {'Alpha Downside': 66.99617760031708,
   'Alpha PT WIC': 88.99617760031708,
   'Alpha Upside': 103.49617760031708},
  'EV/EBITDA': {'Alpha Downside': 65.48686888944266,
   'Alpha PT WIC': 87.48686888944266,
   'Alpha Upside': 101.98686888944266},
  'EV/Sales': {'Alpha Downside': 61.534785667519216,
   'Alpha PT WIC': 83.53478566751922,
   'Alpha Upside': 98.03478566751922},
  'DVD Yield': {'Alpha Downside': 82.42916027655974,
   'Alpha PT WIC': 104.42916027655974,
   'Alpha Upside': 118.92916027655974},
  'FCF Yield': {'Alpha Downside': 68.37146359061748,
   'Alpha PT WIC': 90.37146359061748,
   'Alpha Upside': 104.87146359061748}},
 'overlap_peer_results': {'P/E': {'Alpha Downside': 68.75258258217437,
   'Alpha PT WIC': 90.75258258217437,
   'Alpha Upside': 105.25258258217437},
  'EV/EBITDA': {'Alpha Downside': 66.4496910905316,
   'Alpha PT WIC': 88.4496910905316,
   'Alpha Upside': 102.9496910905316},
  'EV/Sales': {'Alpha Downside': 70.2482728148017

In [21]:
alpha_ticker = "NXST US EQUITY"
lookback_period = 120
unaffect_dt = '2018-11-30'
target_date = "2019-01-08"
analyst_upside = 97.50
fperiod = "1BF"
metrics = ['P/E', "EV/EBITDA", "EV/Sales", 'DVD Yield', 'FCF Yield']
api_host = bbgclient.bbgclient.get_next_available_host()
slicer = dfutils.df_slicer()
unaff_dt = datetime.datetime.strptime(unaffect_dt, '%Y-%m-%d')
price_tgt_dt = datetime.datetime.strptime(target_date, '%Y-%m-%d')
as_of_dt = datetime.datetime.today()
    
peer_ticker_list = bloomberg_peers(alpha_ticker)
    
alpha_historical_mult_df = multiples_df(alpha_ticker, slicer.prev_n_business_days(lookback_period,unaff_dt).strftime('%Y%m%d'), unaff_dt.strftime('%Y%m%d'), api_host, fperiod) # ['CID','Date','Ticker','PX','P/E','EV/EBITDA','EV/Sales','DVD yield','FCF yield']
peer2historical_mult_df = {p:multiples_df(p,slicer.prev_n_business_days(lookback_period,unaff_dt).strftime('%Y%m%d'), unaff_dt.strftime('%Y%m%d'), api_host, fperiod, multiples_to_query=metrics) for p in peer_ticker_list}
alpha_balance_sheet_df_ptd = multiple_underlying_df(alpha_ticker,slicer.prev_n_business_days(100,price_tgt_dt).strftime('%Y%m%d'),price_tgt_dt.strftime('%Y%m%d'),api_host,fperiod = '1BF')
alpha_balance_sheet_df_now = multiple_underlying_df(alpha_ticker,slicer.prev_n_business_days(100,as_of_dt).strftime('%Y%m%d'),as_of_dt.strftime('%Y%m%d'),api_host,fperiod = '1BF')
ticker2short_ticker = {p:p.split(' ')[0] for p in peer_ticker_list+[alpha_ticker]}
    
peer2ptd_multiple = {p:multiples_df(p,slicer.prev_n_business_days(100,price_tgt_dt).strftime('%Y%m%d'),price_tgt_dt.strftime('%Y%m%d'),api_host,fperiod='1BF',multiples_to_query=metrics) for p in peer_ticker_list}
peer2now_multiple = {p:multiples_df(p,slicer.prev_n_business_days(100,as_of_dt).strftime('%Y%m%d'),as_of_dt.strftime('%Y%m%d'),api_host,fperiod='1BF',multiples_to_query=metrics) for p in peer_ticker_list}
    
#Runs the regression on all metrics and find the optimal peer group per metric which is then uses to calculate the downside
rows = []
metric2peer2coeff = {m:{} for m in metrics}
peers_d = {}
for metric in metrics:
    if len(alpha_historical_mult_df[~pd.isnull(alpha_historical_mult_df[metric])])>0:
        m_df = alpha_historical_mult_df[['Date',metric]].rename(columns={metric:ticker2short_ticker[alpha_ticker]})
        for p in peer2historical_mult_df:
            m_df = pd.merge(m_df,peer2historical_mult_df[p][['Date',metric]],how='left',on='Date').rename(columns={metric:ticker2short_ticker[p]})
        peer_tickers = [p for p in peer_ticker_list if len(m_df[~pd.isnull(m_df[p.split(' ')[0]])])>0] # remove peers with all nulls
        #peer_tickers = [p for p in peer_tickers if ~peer2ptd_multiple[p][metric].isnull().all() and ~peer2now_multiple[p][metric].isnull().all()]
        m_ols_df = m_df[[alpha_ticker.split(' ')[0]]+[t.split(' ')[0] for t in peer_tickers]]
        formula = alpha_ticker.split(' ')[0] + '~.' #+ " + ".join([t.split(' ')[0] for t in peer_ticker_list])
        r_df = pandas2ri.py2ri(m_ols_df)
        r_df = stats.na_omit(r_df)
        model = stats.lm(formula, data = r_df)
        optimal_model = stats.step(model, direction = "backward", trace = False)
        summary_results = base.summary(optimal_model)
        summary_results2 = pandas2ri.ri2py(summary_results)
        optimal_peers = base.attr(optimal_model.rx2('terms'), "term.labels")
        optimal_peers2 = []
        for i in optimal_peers:
            for k in peer_tickers:
                if k.startswith(i + ' '):
                    optimal_peers2.append(k)
                elif i.startswith('X') and k.startswith(i[1:] + ' '):
                    optimal_peers2.append(k)
                elif '.' in i:
                    i = i.replace('.', '/')
                    if k.startswith(i + ' '):
                        optimal_peers2.append(k)
                #Add the "__ EQUITY" phrase back into the optimal peer list for Bloomberg search
        peers_d[metric] = optimal_peers2
        optimal_model_coeff = optimal_model.rx2('coefficients')
        optimal_model_coeff_df = pd.DataFrame(columns=['Peer','Coefficient'],data=[(optimal_peers2[count],optimal_model_coeff[count + 1]) for count in range(0,len(optimal_model_coeff.names)-1)])
        optimal_model_coeff_df = optimal_model_coeff_df.append({'Peer': 'Intercept', 'Coefficient': optimal_model_coeff[0]}, ignore_index = True)
        peer2coeff = {p:optimal_model_coeff.rx2(p)[0] for p in optimal_model_coeff.names}
        del peer2coeff["(Intercept)"]
        count = 0
        for k in optimal_peers:
            peer2coeff[optimal_peers2[count]] = peer2coeff.pop(k)
            count = count + 1
        peer2coeff["(Intercept)"] = optimal_model_coeff[0]
        metric2peer2coeff[metric] = peer2coeff
        rows.append(metric)
        #rows[metric] = optimal_model_coeff_df
    else:
        continue
        
df = pd.DataFrame(columns=['Metric'], data = rows)
df['Peers Multiples DataFrame @ Price Target Date'] = df['Metric'].apply(lambda m: pd.DataFrame(columns=['Peer','Multiple'],data=[(p,peer2ptd_multiple[p][m].fillna(0).iloc[-1]) for p in peers_d[m]]))
df['Alpha Implied Multiple @ Price Target Date'] = df['Metric'].apply(lambda m: sum([metric2peer2coeff[m][p]*peer2ptd_multiple[p][m].fillna(0).iloc[-1] for p in peers_d[m]])+ metric2peer2coeff[m]['(Intercept)'])
df['Alpha Balance Sheet DataFrame @ Price Target Date'] =  [alpha_balance_sheet_df_ptd]*len(df)
df['Alpha Unaffected PX @ Price Target Date'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_ptd) for (m,mult) in zip(df['Metric'],df['Alpha Implied Multiple @ Price Target Date'])]

df['Peers Multiples DataFrame @ Now'] = df['Metric'].apply(lambda m: pd.DataFrame(columns=['Peer','Multiple'],data=[(p,peer2now_multiple[p][m].fillna(0).iloc[-1]) for p in peers_d[m]]))
df['Alpha Implied Multiple @ Now'] = df['Metric'].apply(lambda m: sum([metric2peer2coeff[m][p]*peer2now_multiple[p][m].fillna(0).iloc[-1] for p in peers_d[m]])+metric2peer2coeff[m]['(Intercept)'])
df['Alpha Balance Sheet DataFrame @ Now'] =  [alpha_balance_sheet_df_now]*len(df)
df['Alpha Unaffected PX @ Now'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_now) for (m,mult) in zip(df['Metric'],df['Alpha Implied Multiple @ Now'])]

df['Alpha Upside (analyst)'] = analyst_upside
df['Premium'] = (df['Alpha Upside (analyst)'].astype(float)-df['Alpha Unaffected PX @ Price Target Date'].astype(float)).fillna(0).apply(lambda x: max(x,0))

df['Alpha Downside (Adj,weighted)'] = df['Alpha Unaffected PX @ Now'].astype(float)
df['Alpha Upside (Adj,weighted)'] = df['Alpha Unaffected PX @ Now'].astype(float)+df['Premium'].astype(float)

In [20]:
df

,Metric,Peers Multiples DataFrame @ Price Target Date,Alpha Implied Multiple @ Price Target Date,Alpha Balance Sheet DataFrame @ Price Target Date,Alpha Unaffected PX @ Price Target Date,Peers Multiples DataFrame @ Now,Alpha Implied Multiple @ Now,Alpha Balance Sheet DataFrame @ Now,Alpha Unaffected PX @ Now,Alpha Upside (analyst),Premium,"Alpha Downside (Adj,weighted)","Alpha Upside (Adj,weighted)"
0,P/E,Peer Multiple 0 TRCO US EQUIT...,9.335794,Date PX CUR_MKT_CAP EQY_SH_OUT ...,63.520744,Peer Multiple 0 TRCO US EQUIT...,10.096632,Date PX CUR_MKT_CAP EQY_SH_OUT ...,73.069329,97.5,33.979256,73.069329,107.048585
1,EV/EBITDA,Peer Multiple 0 SBGI US EQUIT...,7.783495,Date PX CUR_MKT_CAP EQY_SH_OUT ...,62.234288,Peer Multiple 0 SBGI US EQUIT...,7.808564,Date PX CUR_MKT_CAP EQY_SH_OUT ...,NaN,97.5,35.265712,NaN,NaN
2,EV/Sales,Peer Multiple 0 SBGI US EQUIT...,2.262039,Date PX CUR_MKT_CAP EQY_SH_OUT ...,52.763853,Peer Multiple 0 SBGI US EQUIT...,2.219405,Date PX CUR_MKT_CAP EQY_SH_OUT ...,NaN,97.5,44.736147,NaN,NaN
3,DVD Yield,Peer Multiple 0 SBGI US EQUITY ...,2.019578,Date PX CUR_MKT_CAP EQY_SH_OUT ...,74.273425,Peer Multiple 0 SBGI US EQUITY ...,1.865343,Date PX CUR_MKT_CAP EQY_SH_OUT ...,96.539160,97.5,23.226575,96.539160,119.765736
4,FCF Yield,Peer Multiple 0 SBGI US EQUITY ...,0.147761,Date PX CUR_MKT_CAP EQY_SH_OUT ...,137.133349,Peer Multiple 0 SBGI US EQUITY ...,0.142825,Date PX CUR_MKT_CAP EQY_SH_OUT ...,147.679664,97.5,0.000000,147.679664,147.679664
